In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
# import ydf
import tensorflow_decision_forests as tfdf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs found: {gpus}")
else:
    print("No GPUs found.")

2025-11-08 12:31:51.011462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762605111.265677      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762605111.334225      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


No GPUs found.


2025-11-08 12:32:09.185307: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [2]:
train_df=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_df=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [3]:
# preprocess?
def preprocess(df):
    # def cryoSleep_integer(x):
    #     return x.astype(int)
    # df=df.dropna()
    def normalize_name(x):
        x=str(x)
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    def split_passenger_id(row):
        if pd.isna(row['PassengerId']):
            return pd.Series([None, None, None])
        arr=row['PassengerId'].split('_')
        if len(arr) < 2:
            arr += [None] * (3 - len(arr))
        
        return pd.Series(arr)
    def split_cabin(row):
        if pd.isna(row['Cabin']):
            return pd.Series([None, None, None])  # Handle None case
        arr = row['Cabin'].split('/')
        # Ensure arr has exactly 3 elements
        if len(arr) < 3:
            arr += [None] * (3 - len(arr))  # Pad with None
        return pd.Series(arr)
    def total_spent(row):
        return pd.Series(row['RoomService']+row['FoodCourt']+row['ShoppingMall']+row['Spa']+row['VRDeck'])
        

    # Apply the function and create new columns
    df.fillna({
    'RoomService': df['RoomService'].median(),
    'FoodCourt': df['FoodCourt'].median(),
    'ShoppingMall':df['ShoppingMall'].median(),
    'Spa':df['Spa'].median(),
    'VRDeck':df['VRDeck'].median(),
    # 'Deck':df['Deck'].median()
    }, inplace=True)
    df['Total_Spent']=df.apply(total_spent,axis=1)
    df["Name"] = df["Name"].apply(normalize_name)
    df[['Deck', 'Num', 'Side']] = df.apply(split_cabin, axis=1)
    df[['Group','Place_In_Group']]=df.apply(split_passenger_id,axis=1)
    df['CryoSleep'] = df['CryoSleep'].apply(lambda x: float(x))
    df=df.fillna({'CryoSleep':df['CryoSleep'].mean()})
    df['VIP'] = df['VIP'].apply(lambda x: float(x))

        
    # keep PassengerId and one-hot encode HomePlanet & Destination
    id_col = df['PassengerId']                     # actual column values
    dums = pd.get_dummies(df[['HomePlanet','Destination','Deck','Side']], dummy_na=False)
    
    # drop the original encoded columns, then concat PassengerId + dummies + remaining columns
    remaining = df.drop(columns=['HomePlanet','Destination','PassengerId'])
    df = pd.concat([id_col.reset_index(drop=True), remaining.reset_index(drop=True), dums.reset_index(drop=True)], axis=1)
    
    # convert boolean columns to 0/1
    bool_cols = df.select_dtypes(include='bool').columns
    df[bool_cols] = df[bool_cols].astype(int)
    
    return df
    
train_df=preprocess(train_df)
test_df=preprocess(test_df)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                8693 non-null   object 
 1   CryoSleep                  8693 non-null   float64
 2   Cabin                      8494 non-null   object 
 3   Age                        8514 non-null   float64
 4   VIP                        8490 non-null   float64
 5   RoomService                8693 non-null   float64
 6   FoodCourt                  8693 non-null   float64
 7   ShoppingMall               8693 non-null   float64
 8   Spa                        8693 non-null   float64
 9   VRDeck                     8693 non-null   float64
 10  Name                       8693 non-null   object 
 11  Transported                8693 non-null   int64  
 12  Total_Spent                8693 non-null   float64
 13  Deck                       8494 non-null   objec

In [5]:
train_df['Deck']

0       B
1       F
2       A
3       A
4       F
       ..
8688    A
8689    G
8690    G
8691    E
8692    E
Name: Deck, Length: 8693, dtype: object

In [6]:
train_df.head(5)

,PassengerId,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S
0,0001_01,0.0,B/0/P,39.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
1,0002_01,0.0,F/0/S,24.0,0.0,109.0,9.0,25.0,549.0,44.0,...,0,0,0,0,0,1,0,0,0,1
2,0003_01,0.0,A/0/S,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,...,1,0,0,0,0,0,0,0,0,1
3,0003_02,0.0,A/0/S,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,...,1,0,0,0,0,0,0,0,0,1
4,0004_01,0.0,F/1/S,16.0,0.0,303.0,70.0,151.0,565.0,2.0,...,0,0,0,0,0,1,0,0,0,1


In [7]:

# train_df[bool_cols].fillna()

In [8]:
train_df

,PassengerId,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S
0,0001_01,0.0,B/0/P,39.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
1,0002_01,0.0,F/0/S,24.0,0.0,109.0,9.0,25.0,549.0,44.0,...,0,0,0,0,0,1,0,0,0,1
2,0003_01,0.0,A/0/S,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,...,1,0,0,0,0,0,0,0,0,1
3,0003_02,0.0,A/0/S,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,...,1,0,0,0,0,0,0,0,0,1
4,0004_01,0.0,F/1/S,16.0,0.0,303.0,70.0,151.0,565.0,2.0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0.0,A/98/P,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,...,1,0,0,0,0,0,0,0,1,0
8689,9278_01,1.0,G/1499/S,18.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,1
8690,9279_01,0.0,G/1500/S,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,...,0,0,0,0,0,0,1,0,0,1
8691,9280_01,0.0,E/608/S,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,...,0,0,0,0,1,0,0,0,0,1


In [9]:

input_features=list(train_df.columns)
input_features.remove('Transported')
input_features.remove('PassengerId')
input_features.remove('Cabin')
# input_features=['CryoSleep']
input_features.remove('Name')
input_features.remove('Side')
input_features.remove('Deck')

In [10]:
train_df=train_df.fillna(0)

In [11]:
from sklearn.preprocessing import StandardScaler



# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(len(input_features),)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='ADAM', 
              loss = tf.keras.losses.BinaryFocalCrossentropy(),
              metrics=['accuracy'])

# Prepare the training data
X_train = train_df[input_features].values
y_train = train_df['Transported'].astype(int).values
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# Fit the model
from tensorflow.keras.callbacks import ModelCheckpoint

# checkpoint to save best model by validation accuracy
ckpt = ModelCheckpoint("best_model.weights.h5", monitor="val_accuracy", save_best_only=True, mode="max", save_weights_only=True)

history = model.fit(X_train_scaled, y_train,
                    epochs=30, batch_size=32,
                    validation_split=0.2,
                    callbacks=[ckpt])

# load best weights saved during training
model.load_weights("best_model.weights.h5")

# prepare test and predict
X_test = test_df[input_features].copy()
X_test = X_test.apply(pd.to_numeric, errors='coerce')
X_test = X_test.fillna(X_test.median())
X_test = scaler.transform(X_test)
predictions = model.predict(X_test)


Epoch 1/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7333 - loss: 0.1321 - val_accuracy: 0.7389 - val_loss: 0.1381
Epoch 2/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7997 - loss: 0.1039 - val_accuracy: 0.7320 - val_loss: 0.1582
Epoch 3/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8116 - loss: 0.0997 - val_accuracy: 0.7677 - val_loss: 0.1417
Epoch 4/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8157 - loss: 0.0966 - val_accuracy: 0.7619 - val_loss: 0.1581
Epoch 5/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8282 - loss: 0.0923 - val_accuracy: 0.7412 - val_loss: 0.1690
Epoch 6/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8201 - loss: 0.0954 - val_accuracy: 0.7706 - val_loss: 0.1596
Epoch 7/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8233 - loss: 0.0927 - val_accuracy: 0.7855 - val_loss: 0.1526
Epoch 8/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8317 - loss: 0.0924 - val_accuracy: 0.

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         3,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,245 (165.02 KB)

 Trainable params: 14,081 (55.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 28,164 (110.02 KB)

In [13]:
def prediction_to_kaggle_format(model, df,X, threshold=0.5):
    # Prepare features for predictions using the same input features as used in training
    # X_test = test_df[input_features].values  # Ensure input features are consistent
    proba_survive = model.predict(X, verbose=0)  # Predict probabilities
    
    # Check the shape of proba_survive
    if proba_survive.ndim > 1:
        proba_survive = proba_survive[:, 0]  # Flatten if necessary
    
    # Create the DataFrame for submission
    return pd.DataFrame({
        "PassengerId": df["PassengerId"].values,  # Ensure this is a 1D array
        "Transported": (proba_survive >= threshold).astype(str)  # Convert boolean to string
    })

def make_submission(kaggle_predictions, name):
    path = f"/kaggle/working/{name}.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")

# Generate predictions and make the submission
kaggle_predictions = prediction_to_kaggle_format(model, test_df,X_test)  # Use preprocessed test_df
make_submission(kaggle_predictions,'submission')

# Optional: Check the first few lines of the submission
!head /kaggle/working/submission.csv

Submission exported to /kaggle/working/submission.csv
PassengerId,Transported
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,True
0027_01,True
0029_01,True
0032_01,True
0032_02,True


In [14]:
X_train = train_df[input_features].copy()
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_train = X_train.fillna(X_train.median())
X_train = scaler.transform(X_train)
predictions = model.predict(X_train)

kaggle_predictions = prediction_to_kaggle_format(model, train_df,X_train)
make_submission(kaggle_predictions, 'ensemble')


!head /kaggle/working/ensemble.csv

100/272 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


272/272 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Submission exported to /kaggle/working/ensemble.csv
PassengerId,Transported
0001_01,False
0002_01,True
0003_01,False
0003_02,False
0004_01,False
0005_01,True
0006_01,True
0006_02,True
0007_01,True
